# Predicting direction of stock price index movement using artificial neural networks

## Estudio por Julio Gutiérrez

In [1]:
import numpy as np
import pandas as pd
import talib

Because the daily data for Istambul Stock Exchange National Index 100 couldn't be found, the study is replicated with the IPC daily data from 2006 to 2016.

In [2]:
market = pd.read_csv('data/IPC-06-10y.csv', index_col='Date', parse_dates=True)
market.columns = ['open', 'high', 'low', 'close', 'adjclose', 'volume']
market.tail()

,open,high,low,close,adjclose,volume
Date,,,,,,
2016-12-26,45219.351563,45219.351563,44862.449219,44989.578125,44989.578125,11005500
2016-12-27,44989.578125,45329.261719,44938.738281,45299.671875,45299.671875,88289700
2016-12-28,45301.199219,45619.839844,45257.718750,45563.179688,45563.179688,85768300
2016-12-29,45564.519531,46025.648438,45549.390625,45909.308594,45909.308594,95022800
2016-12-30,45910.441406,45986.289063,45374.410156,45642.898438,45642.898438,81280000


In [96]:
market = market.apply(pd.to_numeric, errors='coerce')
market.dtypes

open        float64
high        float64
low         float64
close       float64
adjclose    float64
volume      float64
dtype: object

Deal with NaN by performing a quadratic interpolation:

In [97]:
market = market.interpolate(method='quadratic')
market[market.close.isnull()]

,open,high,low,close,adjclose,volume
Date,,,,,,


The direction of daily change in the IPC is categorized as `0` or `1`. If the IPC Index at time `t` is higher than that at time `t-1`, direction `t` is `1`. If the IPC Index at time `t` is lower than that at time `t-1`, direction `t` is `0`.

In [98]:
market['prior_close'] = market.close.shift(1)
market['direction'] = np.where(market['close'] > market['prior_close'], 1, 0)
market['future_direction'] = market.direction.shift(-1)
market = market.iloc[:-1]
market['future_direction'] = market.future_direction.apply(int)
market.tail(10)

,open,high,low,close,adjclose,volume,prior_close,direction,future_direction
Date,,,,,,,,,
2016-12-16,45871.769531,45937.109375,45099.730469,45121.390625,45121.390625,377753600.0,45871.960938,0,0
2016-12-19,45121.191406,45265.781250,44875.679688,44895.289063,44895.289063,134561800.0,45121.390625,0,1
2016-12-20,44894.691406,45266.421875,44761.968750,44930.269531,44930.269531,194329900.0,44895.289063,1,1
2016-12-21,44936.789063,45110.808594,44817.300781,44983.031250,44983.031250,138917900.0,44930.269531,1,1
2016-12-22,44982.429688,45041.421875,44568.269531,45008.078125,45008.078125,130553400.0,44983.031250,1,1
2016-12-23,45016.648438,45263.230469,44922.281250,45173.539063,45173.539063,62035500.0,45008.078125,1,0
2016-12-26,45219.351563,45219.351563,44862.449219,44989.578125,44989.578125,11005500.0,45173.539063,0,1
2016-12-27,44989.578125,45329.261719,44938.738281,45299.671875,45299.671875,88289700.0,44989.578125,1,1
2016-12-28,45301.199219,45619.839844,45257.718750,45563.179688,45563.179688,85768300.0,45299.671875,1,1


The sample size by year varies in size

In [99]:
year_samples = market.groupby(market.index.year).count()
year_samples

,open,high,low,close,adjclose,volume,prior_close,direction,future_direction
Date,,,,,,,,,
2006,260,260,260,260,260,260,259,260,260
2007,253,253,253,253,253,253,253,253,253
2008,253,253,253,253,253,253,253,253,253
2009,252,252,252,252,252,252,252,252,252
2010,249,249,249,249,249,249,249,249,249
2011,252,252,252,252,252,252,252,252,252
2012,248,248,248,248,248,248,248,248,248
2013,249,249,249,249,249,249,249,249,249
2014,251,251,251,251,251,251,251,251,251


## Separation of the data set for training, testing and validation

We will take an 70% of the samples of each year to build a training set, a 15% of the samples of each year to build a testing set and finally another 15% to build the validation set.

In [100]:
training_set_frac = 0.7
testing_set_frac = 0.15
validation_set_frac = 0.15

training_set, testing_set, validation_set = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

In [101]:
for ith in range(len(year_samples)):
    year = year_samples.index[ith]
    set_size = year_samples.iloc[ith].close

    training_size = int(set_size * training_set_frac)
    testing_size = int(set_size * testing_set_frac)
    validation_size = int(set_size * validation_set_frac)
    
    year_set = market[market.index.year == year]

    training_set = training_set.append(year_set[:training_size])
    testing_set = testing_set.append(year_set[training_size:training_size + testing_size + 1])
    validation_set = validation_set.append(year_set[training_size + testing_size + 1:])

The training set consists of the first 70% of samples of each year:

In [102]:
training_set.groupby(training_set.index.year).count()

,open,high,low,close,adjclose,volume,prior_close,direction,future_direction
Date,,,,,,,,,
2006,182,182,182,182,182,182,181,182,182
2007,177,177,177,177,177,177,177,177,177
2008,177,177,177,177,177,177,177,177,177
2009,176,176,176,176,176,176,176,176,176
2010,174,174,174,174,174,174,174,174,174
2011,176,176,176,176,176,176,176,176,176
2012,173,173,173,173,173,173,173,173,173
2013,174,174,174,174,174,174,174,174,174
2014,175,175,175,175,175,175,175,175,175


The testing set consists of 15% of samples of each year:

In [103]:
testing_set.groupby(testing_set.index.year).count()

,open,high,low,close,adjclose,volume,prior_close,direction,future_direction
Date,,,,,,,,,
2006,40,40,40,40,40,40,40,40,40
2007,38,38,38,38,38,38,38,38,38
2008,38,38,38,38,38,38,38,38,38
2009,38,38,38,38,38,38,38,38,38
2010,38,38,38,38,38,38,38,38,38
2011,38,38,38,38,38,38,38,38,38
2012,38,38,38,38,38,38,38,38,38
2013,38,38,38,38,38,38,38,38,38
2014,38,38,38,38,38,38,38,38,38


Finally the validation testing set consists of the last 15% of samples of each year:

In [104]:
validation_set.groupby(validation_set.index.year).count()

,open,high,low,close,adjclose,volume,prior_close,direction,future_direction
Date,,,,,,,,,
2006,38,38,38,38,38,38,38,38,38
2007,38,38,38,38,38,38,38,38,38
2008,38,38,38,38,38,38,38,38,38
2009,38,38,38,38,38,38,38,38,38
2010,37,37,37,37,37,37,37,37,37
2011,38,38,38,38,38,38,38,38,38
2012,37,37,37,37,37,37,37,37,37
2013,37,37,37,37,37,37,37,37,37
2014,38,38,38,38,38,38,38,38,38


## Computation of the indicators data

The input to the network is computed data of 10 technical indicators:

- SMA10
- WMA10
- Momentum
- Stochastic K%
- Stochastic D%
- RSI
- MACD
- Larry William's R%
- A/D Oscilator
- CCI

In [105]:
open_ = market.open.values
high = market.high.values
low = market.low.values
close = market.close.values
volume = market.close.values
sample_size = 10

In [106]:
indicators = pd.DataFrame(index=market.index)
indicators.tail()

""
Date
2016-12-23
2016-12-26
2016-12-27
2016-12-28
2016-12-29


In [107]:
indicators['sma'] = talib.SMA(close, timeperiod=sample_size)

In [108]:
indicators['wma'] = talib.WMA(close, timeperiod=sample_size)

In [109]:
indicators['momentum'] = talib.MOM(close, timeperiod=sample_size)

In [110]:
# Revisar configuración, este produce los dos valores %K y %D
slowk, slowd = talib.STOCH(high, low, close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
indicators['stochk'] = slowk
indicators['stochd'] = slowd

In [111]:
indicators['rsi'] = talib.RSI(close, timeperiod=sample_size)

In [112]:
macd, macdsignal, macdhist = talib.MACD(close, fastperiod=12, slowperiod=26, signalperiod=9)
indicators['macd'] = macd
indicators['macdsig'] = macdsignal
indicators['macdhist'] = macdhist
macd

array([      nan,       nan,       nan, ..., -245.3397, -205.5576,
       -144.4354])

In [113]:
indicators['williamsr'] = talib.WILLR(high, low, close, timeperiod=sample_size)

In [114]:
indicators['ad'] = talib.AD(high, low, close, volume)

In [115]:
indicators['cci'] = talib.CCI(high, low, close, timeperiod=sample_size)

Now is necessary to merge the indicators columns in to the splitted sets (training, testing, validation).

In [116]:
indicators.tail()

,sma,wma,momentum,stochk,stochd,rsi,macd,macdsig,macdhist,williamsr,ad,cci
Date,,,,,,,,,,,,
2016-12-23,45598.780469,45253.933168,-1186.691406,44.610475,23.614853,44.310767,-256.289148,-237.628412,-18.660736,-77.354980,7.236627e+06,-49.451493
2016-12-26,45406.391406,45143.169105,-1923.890625,59.724054,40.889470,41.357140,-265.717661,-243.246262,-22.471399,-84.237532,7.223688e+06,-51.078596
2016-12-27,45249.334766,45123.765554,-1570.566406,81.051271,61.795267,47.865832,-245.339727,-243.664955,-1.674772,-68.653223,7.262123e+06,-18.235332
2016-12-28,45183.598828,45180.828267,-657.359375,83.689918,74.821748,52.810962,-205.557649,-236.043494,30.485845,-40.314110,7.293428e+06,74.061587
2016-12-29,45187.333594,45312.775497,37.347656,93.573939,86.105043,58.549333,-144.435394,-217.721874,73.286480,-7.982814,7.316908e+06,188.124289


In [117]:
training_set = training_set.join(indicators)
testing_set = testing_set.join(indicators)
validation_set = validation_set.join(indicators)

In [118]:
training_set = training_set.iloc[33:]  # We discard first rows because some indicators don't have values
training_set.head()

,open,high,low,close,adjclose,volume,prior_close,direction,future_direction,sma,...,momentum,stochk,stochd,rsi,macd,macdsig,macdhist,williamsr,ad,cci
Date,,,,,,,,,,,,,,,,,,,,,
2006-02-16,18169.160156,18519.990234,18169.160156,18457.240234,18457.240234,172319800.0,18169.160156,1,1,18412.074137,...,-603.140625,51.196426,29.363275,50.098010,-120.317464,-28.490966,-91.826498,-47.876942,119981.571784,-12.672539
2006-02-17,18457.650391,18507.300781,18407.349609,18480.779297,18480.779297,98501200.0,18457.240234,1,1,18373.934098,...,-381.400391,76.272299,50.229793,50.736371,-106.915646,-44.175902,-62.739744,-42.578767,128654.808405,21.615055
2006-02-20,18480.779297,18574.269531,18437.970703,18542.580078,18542.580078,7561400.0,18480.779297,1,0,18344.861133,...,-290.729657,93.376167,73.614964,52.508623,-90.267257,-53.394173,-36.873084,-37.446536,138575.095416,50.158387
2006-02-21,18562.960938,18693.300781,18493.070313,18497.380859,18497.380859,103781400.0,18542.580078,0,1,18328.093164,...,-167.679688,87.452820,85.700429,51.017169,-79.800597,-58.675458,-21.125139,-41.200094,120874.134921,74.384611
2006-02-22,18495.220703,18780.460938,18495.220703,18780.460938,18780.460938,129935000.0,18497.380859,1,1,18365.115234,...,370.220704,89.241656,90.023548,59.101163,-48.108927,-56.562152,8.453224,-0.000000,139654.595859,111.211483


In [119]:
# macdsig y macdhist no estan en el paper pero igual se auto-calcularon (no se agregan)
cols = ['sma', 'wma', 'momentum', 'stochk', 'stochd', 'rsi', 'macd', 'williamsr', 'ad', 'cci']

Xtrain = training_set[cols].values
Ytrain = training_set['future_direction'].values

Xtest = testing_set[cols].values
Ytest = testing_set['future_direction'].values

Xval = validation_set[cols].values
Yval = validation_set['future_direction'].values

print('Training shape:', Xtrain.shape, Ytrain.shape)
print('Testing shape:', Xtest.shape, Ytest.shape)
print('Validation shape:', Xval.shape, Yval.shape)

Training shape: (1901, 10) (1901,)
Testing shape: (420, 10) (420,)
Validation shape: (415, 10) (415,)


## Normalization of the data

The original data was scaled into the range of `[-1, 1]`

In [120]:
from sklearn.preprocessing import MinMaxScaler

np.set_printoptions(precision=4)
scaler = MinMaxScaler(feature_range=(-1, 1))

Xtrain_norm = scaler.fit_transform(Xtrain)
Xtest_norm = scaler.fit_transform(Xtest)
Xval_norm = scaler.fit_transform(Xval)

print('Xtrain_norm:\n', Xtrain_norm[:5, :])
print('Xtest_norm:\n', Xtest_norm[:5, :])
print('Xval_norm:\n', Xval_norm[:5, :])

Xtrain_norm:
 [[-0.9366 -0.9386 -0.1381 -0.0126 -0.4547 -0.0941 -0.1342 -0.0368 -1.
  -0.028 ]
 [-0.9391 -0.9378 -0.0792  0.4821 -0.0182 -0.0767 -0.1196  0.0611 -0.9976
   0.0851]
 [-0.941  -0.9358 -0.0551  0.8195  0.4709 -0.0284 -0.1014  0.156  -0.9948
   0.1793]
 [-0.9421 -0.934  -0.0225  0.7026  0.7237 -0.069  -0.09    0.0866 -0.9998
   0.2592]
 [-0.9397 -0.9287  0.1203  0.7379  0.8142  0.1512 -0.0554  0.848  -0.9945
   0.3807]]
Xtest_norm:
 [[-0.8576 -0.8312  0.3988  0.2571 -0.2206  0.314   0.6176  0.6579 -1.
   0.2934]
 [-0.8556 -0.8277  0.4716  0.7719  0.2607  0.3227  0.6262  0.6852 -0.9992
   0.3345]
 [-0.8532 -0.822   0.4893  0.8545  0.6437  0.4557  0.6447  0.8474 -0.9965
   0.4419]
 [-0.8512 -0.8152  0.4705  0.8073  0.8376  0.5219  0.6646  0.7793 -0.9967
   0.5003]
 [-0.8485 -0.8089  0.5007  0.7408  0.8266  0.5131  0.6774  0.7687 -0.9929
   0.3743]]
Xval_norm:
 [[-0.7121 -0.6883  0.2368  0.453   0.5728  0.5168  0.5305  0.6879 -0.9977
   0.4736]
 [-0.708  -0.6789  0.2499  0.372

Compute distribution of Y classes in the datasets:

In [121]:
Ytrain1 = Ytrain.sum()
Ytrain0 = len(Ytrain) - Ytrain1
Ytrain1p = Ytrain1 / len(Ytrain) * 100
Ytrain0p = Ytrain0 / len(Ytrain) * 100

Ytest1 = Ytest.sum()
Ytest0 = len(Ytest) - Ytest1
Ytest1p = Ytest1 / len(Ytest) * 100
Ytest0p = Ytest0 / len(Ytest) * 100

Yval1 = Yval.sum()
Yval0 = len(Yval) - Yval1
Yval1p = Yval1 / len(Yval) * 100
Yval0p = Yval0 / len(Yval) * 100

print('Train examples: {}     | Y = 1: {} ({:.2f}%)| Y = 0: {} ({:.2f}%)'.format(len(Xtrain_norm), Ytrain1, Ytrain1p, Ytrain0, Ytrain0p))
print('Test examples: {}       | Y = 1: {} ({:.2f}%) | Y = 0: {} ({:.2f}%)'.format(len(Xtest_norm), Ytest1, Ytest1p, Ytest0, Ytest0p))
print('Validation examples: {} | Y = 1: {} ({:.2f}%) | Y = 0: {} ({:.2f}%)'.format(len(Xval_norm), Yval1, Yval1p, Yval0, Yval0p))
print('Total observations:', len(Xtrain) + len(Xtest) + len(Xval))

Train examples: 1901     | Y = 1: 1012 (53.24%)| Y = 0: 889 (46.76%)
Test examples: 420       | Y = 1: 229 (54.52%) | Y = 0: 191 (45.48%)
Validation examples: 415 | Y = 1: 221 (53.25%) | Y = 0: 194 (46.75%)
Total observations: 2736


Tested parameters:
- Number of neurons [10, 20, ..., 100]
- Epochs (max_iter): [1000, 2000, 10000]
- Momentum constant: [0.1, 0.2, ..., 0.9]
- Learning rate: 0.1

## Training the model (single)

In [133]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(
    hidden_layer_sizes=(30, 30, 30),
    max_iter=5000,
    momentum=0.7,
    learning_rate_init=0.1,
    activation='logistic',
    shuffle=False)

mlp.fit(Xtrain_norm, Ytrain)

MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(30, 30, 30), learning_rate='constant',
       learning_rate_init=0.1, max_iter=5000, momentum=0.7,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=False, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [134]:
predictions = mlp.predict(Xtest_norm)

In [135]:
print('Score: {:.2f}%'.format(mlp.score(Xtest_norm, Ytest) * 100))

Score: 50.95%


In [136]:
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support
pr = confusion_matrix(Ytest, predictions)
pr

array([[ 55, 136],
       [ 70, 159]])

In [137]:
from IPython.display import Markdown

true_negatives = pr[0][0]
false_negatives = pr[1][0]
false_positives = pr[0][1]
true_positives = pr[1][1]

Markdown("""
- **True negatives** (_were 0 and predicted as 0_): **{tn}**
- **False negatives** (_were 1 but predicted them as 0_): **{fn}**
- **False positives** (_were 0 but predicted them as 1_): **{fp}**
- **True positives** (_were 1 and predicted as 1_): **{tp}**
""".format(tn=true_negatives, fn=false_negatives, fp=false_positives, tp=true_positives))


- **True negatives** (_were 0 and predicted as 0_): **55**
- **False negatives** (_were 1 but predicted them as 0_): **70**
- **False positives** (_were 0 but predicted them as 1_): **136**
- **True positives** (_were 1 and predicted as 1_): **159**


- _Precision = tp / (tp + fp)_: From all predicted values which ratio was actually well predicted.
- _Recal = tp / (tp + fn)_: From all true values which ratio was actually well predicted.
- _F1 score_: The higher the value the better use of the precision & recall tradeof.
- _Support_: No. of observations of each class in the testing set.

In [138]:
print(classification_report(Ytest, predictions))

             precision    recall  f1-score   support

          0       0.44      0.29      0.35       191
          1       0.54      0.69      0.61       229

avg / total       0.49      0.51      0.49       420



In [139]:
results = precision_recall_fscore_support(Ytest, predictions)
results

(array([ 0.44 ,  0.539]),
 array([ 0.288 ,  0.6943]),
 array([ 0.3481,  0.6069]),
 array([191, 229]))

In [140]:
correctly_predicted = true_positives + true_negatives
badly_predicted = false_positives + false_negatives
print('Accuracy: {:.2f}%'.format(correctly_predicted / (correctly_predicted + badly_predicted) * 100))

Accuracy: 50.95%


## Training the model (multi)

In [280]:
mlp = MLPClassifier(learning_rate_init=0.1, activation='logistic')

epoch = np.linspace(1000, 10000, 10, dtype=int)
neurons = np.linspace(10, 100, 10, dtype=int)
momentum = np.linspace(.1, .9, 9)

In [281]:
from itertools import product
comparative = {}

for params in product(epoch, neurons, momentum):
    ep, n, mc = params
    
    mlp.set_params(max_iter=ep, hidden_layer_sizes=(n,), momentum=mc)
    mlp.fit(Xtrain_norm, Ytrain)
    comparative[params] = mlp.score(Xtest_norm, Ytest)

In [282]:
print('Lowest score in the 900 parameter combinations: {:.2f}%'.format(min(comparative.values()) * 100))
print('Max score in the 900 parameter combinations: {:.2f}%'.format(max(comparative.values()) * 100))

Lowest score in the 900 parameter combinations: 43.77%
Max score in the 900 parameter combinations: 57.65%


In [283]:
import heapq
key_params = heapq.nlargest(5, comparative, key=comparative.get) # Get top 5 by best score

for key in key_params:
    print('Epoch: {}  |  Neurons: {}  |  MC: {:.1f}  |  Score: {:.4f}%'.format(*key, comparative[key] * 100))

Epoch: 10000  |  Neurons: 50  |  MC: 0.1  |  Score: 57.6512%
Epoch: 7000  |  Neurons: 40  |  MC: 0.2  |  Score: 56.9395%
Epoch: 2000  |  Neurons: 30  |  MC: 0.3  |  Score: 56.2278%
Epoch: 9000  |  Neurons: 70  |  MC: 0.4  |  Score: 56.2278%
Epoch: 4000  |  Neurons: 100  |  MC: 0.2  |  Score: 55.8719%


## Analyzing performance in sets of best score parameters

### Calculating yearly scores for the best 1st set of parameters

In [284]:
ep, n, mc = key_params[0]  # 1st set of best parameters

mlp = MLPClassifier(
    learning_rate_init=0.1,
    activation='logistic',
    max_iter=ep,
    hidden_layer_sizes=(n,),
    momentum=mc)

mlp.fit(Xtrain_norm, Ytrain)
Ypred_train = mlp.predict(Xtrain_norm)  # First analysis predicts same training set
Ypred_test = mlp.predict(Xtest_norm)  # Second analysis predicts testing set

In [285]:
train_classification = pd.DataFrame({'Ytrain': Ytrain, 'Ypred_train': Ypred_train}, index=training_set.index)
train_classification.head()

,Ypred_train,Ytrain
Date,,
2007-02-21,0,0
2007-02-22,0,0
2007-02-23,0,0
2007-02-26,0,0
2007-02-27,0,1


In [286]:
test_classification = pd.DataFrame({'Ytest': Ytest, 'Ypred_test': Ypred_test}, index=testing_set.index)
test_classification.head()

,Ypred_test,Ytest
Date,,
2007-10-18,0,0
2007-10-19,0,1
2007-10-22,0,1
2007-10-23,0,0
2007-10-24,0,1


In [287]:
train_benchmark = train_classification['Ytrain'] == train_classification['Ypred_train']
test_benchmark = test_classification['Ytest'] == test_classification['Ypred_test']

train_score = train_benchmark.groupby(train_benchmark.index.year).apply(lambda s: s[s == True].count() / s.count()).rename('train_score')
test_score = test_benchmark.groupby(test_benchmark.index.year).apply(lambda s: s[s == True].count() / s.count()).rename('test_score')

In [288]:
benchmark_one = pd.concat([train_score, test_score], axis=1)
benchmark_one

,train_score,test_score
Date,,
2007,0.449102,0.615385
2008,0.485149,0.615385
2009,0.522388,0.384615
2010,0.497512,0.307692
2011,0.482587,0.576923
2012,0.450000,0.653846
2013,0.442786,0.423077
2014,0.457711,0.269231
2015,0.547264,0.576923


In [289]:
print('Train average: {}\nTest average: {}'.format(
    benchmark_one['train_score'].mean(),
    benchmark_one['test_score'].mean()))

Train average: 0.4890849582467096
Test average: 0.4795204795204796


### Calculating yearly scores for the best 2nd set of parameters

In [290]:
ep, n, mc = key_params[1]  # 2nd set of best parameters

mlp = MLPClassifier(
    learning_rate_init=0.1,
    activation='logistic',
    max_iter=ep,
    hidden_layer_sizes=(n,),
    momentum=mc)

mlp.fit(Xtrain_norm, Ytrain)
Ypred_train = mlp.predict(Xtrain_norm)  # First analysis predicts same training set
Ypred_test = mlp.predict(Xtest_norm)  # Second analysis predicts testing set

In [291]:
train_classification = pd.DataFrame({'Ytrain': Ytrain, 'Ypred_train': Ypred_train}, index=training_set.index)
test_classification = pd.DataFrame({'Ytest': Ytest, 'Ypred_test': Ypred_test}, index=testing_set.index)

In [292]:
train_benchmark = train_classification['Ytrain'] == train_classification['Ypred_train']
test_benchmark = test_classification['Ytest'] == test_classification['Ypred_test']

train_score = train_benchmark.groupby(train_benchmark.index.year).apply(lambda s: s[s == True].count() / s.count()).rename('train_score')
test_score = test_benchmark.groupby(test_benchmark.index.year).apply(lambda s: s[s == True].count() / s.count()).rename('test_score')

In [293]:
benchmark_two = pd.concat([train_score, test_score], axis=1)
benchmark_two

,train_score,test_score
Date,,
2007,0.550898,0.461538
2008,0.500000,0.461538
2009,0.537313,0.615385
2010,0.537313,0.615385
2011,0.527363,0.461538
2012,0.550000,0.461538
2013,0.582090,0.615385
2014,0.577114,0.730769
2015,0.507463,0.384615


In [294]:
print('Train average: {}\nTest average: {}'.format(
    benchmark_two['train_score'].mean(),
    benchmark_two['test_score'].mean()))

Train average: 0.5412738319930409
Test average: 0.5466200466200467


### Calculating yearly scores for the best 3rd set of parameters

In [295]:
ep, n, mc = key_params[2]  # 3rd set of best parameters

mlp = MLPClassifier(
    learning_rate_init=0.1,
    activation='logistic',
    max_iter=ep,
    hidden_layer_sizes=(n,),
    momentum=mc)

mlp.fit(Xtrain_norm, Ytrain)
Ypred_train = mlp.predict(Xtrain_norm)  # First analysis predicts same training set
Ypred_test = mlp.predict(Xtest_norm)  # Second analysis predicts testing set

In [296]:
train_classification = pd.DataFrame({'Ytrain': Ytrain, 'Ypred_train': Ypred_train}, index=training_set.index)
test_classification = pd.DataFrame({'Ytest': Ytest, 'Ypred_test': Ypred_test}, index=testing_set.index)

In [297]:
train_benchmark = train_classification['Ytrain'] == train_classification['Ypred_train']
test_benchmark = test_classification['Ytest'] == test_classification['Ypred_test']

train_score = train_benchmark.groupby(train_benchmark.index.year).apply(lambda s: s[s == True].count() / s.count()).rename('train_score')
test_score = test_benchmark.groupby(test_benchmark.index.year).apply(lambda s: s[s == True].count() / s.count()).rename('test_score')

In [298]:
benchmark_three = pd.concat([train_score, test_score], axis=1)
benchmark_three

,train_score,test_score
Date,,
2007,0.550898,0.461538
2008,0.500000,0.461538
2009,0.537313,0.615385
2010,0.537313,0.615385
2011,0.527363,0.461538
2012,0.550000,0.461538
2013,0.582090,0.615385
2014,0.577114,0.730769
2015,0.517413,0.384615


In [299]:
print('Train average: {:.2f}%\nTest average: {:.2f}%'.format(
    benchmark_three['train_score'].mean() * 100,
    benchmark_three['test_score'].mean() * 100))

Train average: 54.59%
Test average: 52.93%


In [244]:
# Later: Also test various values of learning_rate, hidden_layers and activation function.
# Later: train the network X times for each parameters to have statistical significance
# Later: train the network with multiple market data to see how it behaves in other markets
# Later: Backtest a strategy using the prediction results to get a set of trade results
# Later: Analyze trade results with CDF and Montecarlo